Pandas -> 1 - 5Gb >>>  Chunksize -> 5 - 30Gb

Dask -> 30 - 200Gb

Spark(Scala), PySpark API -> 1000Gb ~ 1Tb, 1000Tb ~ 1Pb 


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/csv/car_evaluation.csv', header=None)
# df = pd.read_csv('../data/csv/us_companies.csv', chunksize=1000)
df.head()

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [4]:
df.columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']

In [5]:
df.describe()

,buying,maint,doors,persons,lug_boot,safety,class
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,vhigh,vhigh,2,2,small,low,unacc
freq,432,432,432,576,576,576,1210


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   class     1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [7]:
df["doors"].replace({"5more":6}, inplace=True)
df["persons"].replace({"more":10}, inplace=True)

df.to_csv("../data/csv/car_evaluation_processed.csv")

In [8]:
import dask.array as da
a = da.random.randint(10,100,100000000).reshape(1000000,100)
a.compute()

array([[67, 13, 55, ..., 56, 38, 46],
       [15, 10, 11, ..., 97, 25, 73],
       [61, 32, 49, ..., 36, 89, 15],
       ...,
       [22, 54, 86, ..., 56, 90, 37],
       [24, 72, 75, ..., 76, 29, 85],
       [12, 82, 12, ..., 22, 79, 21]])

In [9]:
import dask.dataframe as dd

In [10]:
# data = dd.read_csv("../data/csv/car_evaluation.csv", dtype={'buying': str, "maint": str, "num_doors": int, "num_person": int,"lug_boot": str, "safety": str, "decision": str})
# data = dd.read_csv("../data/csv/car_evaluation.csv", header=None)
data = dd.read_csv("../data/csv/car_evaluation_processed.csv")
data.head()
#data.compute()

,Unnamed: 0,buying,maint,doors,persons,lug_boot,safety,class
0,0,vhigh,vhigh,2,2,small,low,unacc
1,1,vhigh,vhigh,2,2,small,med,unacc
2,2,vhigh,vhigh,2,2,small,high,unacc
3,3,vhigh,vhigh,2,2,med,low,unacc
4,4,vhigh,vhigh,2,2,med,med,unacc


In [11]:
import pyspark as spark
from pyspark import SparkContext, SQLContext

In [12]:
try:
    sc = SparkContext()
except:
    print("Spark Context is already running")
s = SQLContext(sc)

/home/serendipita/Documents/utec/bigdata/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [13]:
# data = s.read.options(header=True).csv("../data/csv/car_evaluation.csv")
data = s.read.options(header=True).csv("../data/csv/car_evaluation_processed.csv")

In [14]:
data.show(5)

+---+------+-----+-----+-------+--------+------+-----+
|_c0|buying|maint|doors|persons|lug_boot|safety|class|
+---+------+-----+-----+-------+--------+------+-----+
|  0| vhigh|vhigh|    2|      2|   small|   low|unacc|
|  1| vhigh|vhigh|    2|      2|   small|   med|unacc|
|  2| vhigh|vhigh|    2|      2|   small|  high|unacc|
|  3| vhigh|vhigh|    2|      2|     med|   low|unacc|
|  4| vhigh|vhigh|    2|      2|     med|   med|unacc|
+---+------+-----+-----+-------+--------+------+-----+
only showing top 5 rows



In [15]:
data.select(['maint', 'buying']).show()

+-----+------+
|maint|buying|
+-----+------+
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
|vhigh| vhigh|
+-----+------+
only showing top 20 rows

